In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [2]:
def quadrants384_to_96(data):
    data = data.copy()
    
    quad1 = []
    for row in np.arange(0, 16, 2): 
        for col in np.arange(0, 23, 2):
            quad1.append(data.values[row, col])


    QUAD1_96 = pd.DataFrame(np.array(quad1).reshape([8, 12])) # 96
    QUAD1_96.columns = np.linspace(1, 12, 12, dtype = np.int)
    QUAD1_96['Row'] = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
    QUAD1_96.set_index(['Row'], inplace = True)

    quad2 = []
    for row in np.arange(0, 16, 2): 
        for col in np.arange(1, 24, 2):
            quad2.append(data.values[row, col])


    QUAD2_96 = pd.DataFrame(np.array(quad2).reshape([8, 12])) # 96
    QUAD2_96.columns = np.linspace(1, 12, 12, dtype = np.int)
    QUAD2_96['Row'] = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
    QUAD2_96.set_index(['Row'], inplace = True)

    quad3 = []
    for row in np.arange(1, 17, 2): 
        for col in np.arange(0, 23, 2):
            quad3.append(data.values[row, col])


    QUAD3_96 = pd.DataFrame(np.array(quad3).reshape([8, 12])) # 96
    QUAD3_96.columns = np.linspace(1, 12, 12, dtype = np.int)
    QUAD3_96['Row'] = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
    QUAD3_96.set_index(['Row'], inplace = True)

    quad4 = []
    for row in np.arange(1, 17, 2): 
        for col in np.arange(1, 24, 2):
            quad4.append(data.values[row, col])


    QUAD4_96 = pd.DataFrame(np.array(quad4).reshape([8, 12])) # 96
    QUAD4_96.columns = np.linspace(1, 12, 12, dtype = np.int)
    QUAD4_96['Row'] = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
    QUAD4_96.set_index(['Row'], inplace = True)

    return QUAD1_96, QUAD2_96, QUAD3_96, QUAD4_96


def checkerSetUp(FAMdf, REDdf, input384, sortBy):
    FAMdf = FAMdf.copy()
    REDdf = REDdf.copy()
    
    if input384 == True:
        setRange = 25
    else:
        setRange = 13
    
    wellLocation = []
    wellLocationRow = []
    wellLocationCol = []
    FAM_CT = []
    CalRed_CT = []
    
    specimen_result = []
    repeat_list = []
    for index, row in FAMdf.iterrows():  
            for i in range(1, setRange): 
                FAMValue = FAMdf.loc[index,i]
                
                RedValue = REDdf.loc[index,i]
                FAM_CT.append(FAMValue)
                CalRed_CT.append(RedValue)
                
                
                wellLocationRow.append(index)
                wellLocationCol.append(i)
                wellLocation.append(index + str(i))
                
                if np.isnan(FAMValue): 
                    specimen_result.append('Negative')
                    repeat_list.append('N/A')
                elif FAMValue >= 36 and FAMValue < 40:
                    specimen_result.append('REPEAT')
                    repeat_list.append('REPEAT')
                elif FAMValue >= 40: 
                    specimen_result.append('Negative')
                    repeat_list.append('N/A')
                else:
                    specimen_result.append('Positive')
                    repeat_list.append('N/A')
                
    checker_dict = {
        'Well': wellLocation,
        'CT Value SARS-CoV-2': FAM_CT,
        'CT Value RNASE P': CalRed_CT,
        'Well row': wellLocationRow,
        'Well col': wellLocationCol,
        'Result': specimen_result,
        'REPEAT Ct VALUE SARS-CoV-2': repeat_list,
        'REPEAT Ct VALUE RNASE P': repeat_list,
        'REPEAT RESULT': repeat_list
    }
    
    
    df = pd.DataFrame(checker_dict).sort_values(by = sortBy).\
            fillna('N/A')[['Well', 'CT Value SARS-CoV-2', 'CT Value RNASE P',
                          'Result', 'REPEAT Ct VALUE SARS-CoV-2',
                          'REPEAT Ct VALUE RNASE P', 'REPEAT RESULT']]
    return df
    

In [3]:
def checkerBoardValidationMethod(file: str, input384 = False, sortBy = ['Well row','Well col'],
                                quadrants384_to_96Method = False): # do RED and FAM together eventually
    
   
    FAMdf = pd.read_excel(file)
    FAMdf.rename(columns={'Unnamed: 0':'Rows'}, inplace=True)
    FAMdf.set_index('Rows', inplace = True)
    FAMdf = FAMdf[FAMdf['Unnamed: 1'] == 'Cq'].loc[:, FAMdf.columns != 'Unnamed: 1']
    FAMdf.columns = FAMdf.columns.astype('int64')


    REDdf = pd.read_excel(file,sheet_name = 1)
    REDdf.rename(columns={'Unnamed: 0':'Rows'}, inplace=True)
    REDdf.set_index('Rows', inplace = True)
    REDdf = REDdf[REDdf['Unnamed: 1'] == 'Cq'].loc[:, REDdf.columns != 'Unnamed: 1']
    REDdf.columns = REDdf.columns.astype('int64')
   

    if quadrants384_to_96Method == False:
        return checkerSetUp(FAMdf, REDdf, input384, sortBy)
    
    elif quadrants384_to_96Method == True:
        sarsQUAD1_96, sarsQUAD2_96, sarsQUAD3_96, sarsQUAD4_96 = quadrants384_to_96(FAMdf)
        redQUAD1_96, redQUAD2_96, redQUAD3_96, redQUAD4_96 = quadrants384_to_96(REDdf)
        
        
        dfs_96 = []
        for sars, red in zip([sarsQUAD1_96, sarsQUAD2_96, sarsQUAD3_96, sarsQUAD4_96], 
                            [redQUAD1_96, redQUAD2_96, redQUAD3_96, redQUAD4_96]):
            dfs_96.append(checkerSetUp(sars, red, input384 = False, sortBy = sortBy))
        return dfs_96

Displaying 384 plate data

In [4]:
checkerBoardValidationMethod('RE-CHECKER 384 BETH MM289 BR30 02-17-21 -  Quantification Plate View Results.xlsx',
                            input384=True)#.to_excel('Beth384_checkerPQ.xlsx')

,Well,CT Value SARS-CoV-2,CT Value RNASE P,Result,REPEAT Ct VALUE SARS-CoV-2,REPEAT Ct VALUE RNASE P,REPEAT RESULT
0,A1,N/A,N/A,Negative,N/A,N/A,N/A
1,A2,N/A,N/A,Negative,N/A,N/A,N/A
2,A3,20.6485,25.0585,Positive,N/A,N/A,N/A
3,A4,22.0374,25.6399,Positive,N/A,N/A,N/A
4,A5,N/A,25.725,Negative,N/A,N/A,N/A
5,A6,N/A,26.7815,Negative,N/A,N/A,N/A
6,A7,20.6313,25.4479,Positive,N/A,N/A,N/A
7,A8,22.4339,25.634,Positive,N/A,N/A,N/A
8,A9,N/A,25.8496,Negative,N/A,N/A,N/A
9,A10,N/A,25.8127,Negative,N/A,N/A,N/A


Displaying 384 data in four 96 quadrants. order is set by row.

In [5]:
for i in checkerBoardValidationMethod('RE-CHECKER 384 BETH MM289 BR30 02-17-21 -  Quantification Plate View Results.xlsx',
                            input384=True, quadrants384_to_96Method = True):
    display(i)

,Well,CT Value SARS-CoV-2,CT Value RNASE P,Result,REPEAT Ct VALUE SARS-CoV-2,REPEAT Ct VALUE RNASE P,REPEAT RESULT
0,A1,N/A,N/A,Negative,N/A,N/A,N/A
1,A2,20.6485,25.0585,Positive,N/A,N/A,N/A
2,A3,N/A,25.725,Negative,N/A,N/A,N/A
3,A4,20.6313,25.4479,Positive,N/A,N/A,N/A
4,A5,N/A,25.8496,Negative,N/A,N/A,N/A
5,A6,20.3483,25.5068,Positive,N/A,N/A,N/A
6,A7,N/A,26.6943,Negative,N/A,N/A,N/A
7,A8,20.7303,25.8064,Positive,N/A,N/A,N/A
8,A9,N/A,25.9802,Negative,N/A,N/A,N/A
9,A10,20.8427,25.7533,Positive,N/A,N/A,N/A


,Well,CT Value SARS-CoV-2,CT Value RNASE P,Result,REPEAT Ct VALUE SARS-CoV-2,REPEAT Ct VALUE RNASE P,REPEAT RESULT
0,A1,N/A,N/A,Negative,N/A,N/A,N/A
1,A2,22.0374,25.6399,Positive,N/A,N/A,N/A
2,A3,N/A,26.7815,Negative,N/A,N/A,N/A
3,A4,22.4339,25.634,Positive,N/A,N/A,N/A
4,A5,N/A,25.8127,Negative,N/A,N/A,N/A
5,A6,22.5473,26.2756,Positive,N/A,N/A,N/A
6,A7,38.7971,26.138,REPEAT,REPEAT,REPEAT,REPEAT
7,A8,21.5864,25.6614,Positive,N/A,N/A,N/A
8,A9,N/A,26.2568,Negative,N/A,N/A,N/A
9,A10,21.6331,25.7792,Positive,N/A,N/A,N/A


,Well,CT Value SARS-CoV-2,CT Value RNASE P,Result,REPEAT Ct VALUE SARS-CoV-2,REPEAT Ct VALUE RNASE P,REPEAT RESULT
0,A1,N/A,N/A,Negative,N/A,N/A,N/A
1,A2,20.6237,25.1461,Positive,N/A,N/A,N/A
2,A3,N/A,25.4687,Negative,N/A,N/A,N/A
3,A4,20.6387,25.5241,Positive,N/A,N/A,N/A
4,A5,N/A,25.9091,Negative,N/A,N/A,N/A
5,A6,20.4302,25.684,Positive,N/A,N/A,N/A
6,A7,N/A,26.1777,Negative,N/A,N/A,N/A
7,A8,20.6646,25.7711,Positive,N/A,N/A,N/A
8,A9,N/A,25.9795,Negative,N/A,N/A,N/A
9,A10,20.619,25.4721,Positive,N/A,N/A,N/A


,Well,CT Value SARS-CoV-2,CT Value RNASE P,Result,REPEAT Ct VALUE SARS-CoV-2,REPEAT Ct VALUE RNASE P,REPEAT RESULT
0,A1,N/A,N/A,Negative,N/A,N/A,N/A
1,A2,20.5913,25.3497,Positive,N/A,N/A,N/A
2,A3,N/A,25.7373,Negative,N/A,N/A,N/A
3,A4,20.579,25.3553,Positive,N/A,N/A,N/A
4,A5,N/A,25.9768,Negative,N/A,N/A,N/A
5,A6,20.638,25.9827,Positive,N/A,N/A,N/A
6,A7,N/A,26.1506,Negative,N/A,N/A,N/A
7,A8,20.6994,25.8874,Positive,N/A,N/A,N/A
8,A9,N/A,25.9178,Negative,N/A,N/A,N/A
9,A10,20.4438,25.1412,Positive,N/A,N/A,N/A


Displaying 384 data in four 96 quadrants. order is set by column.

In [6]:
for i, cb in enumerate(checkerBoardValidationMethod('RE-CHECKER 384 BETH MM289 BR30 02-17-21 -  Quantification Plate View Results.xlsx',
                            input384=True, quadrants384_to_96Method = True, sortBy=['Well col', 'Well row'])):
    display(cb)

,Well,CT Value SARS-CoV-2,CT Value RNASE P,Result,REPEAT Ct VALUE SARS-CoV-2,REPEAT Ct VALUE RNASE P,REPEAT RESULT
0,A1,N/A,N/A,Negative,N/A,N/A,N/A
12,B1,29.9555,30.4108,Positive,N/A,N/A,N/A
24,C1,N/A,N/A,Negative,N/A,N/A,N/A
36,D1,29.7362,30.4707,Positive,N/A,N/A,N/A
48,E1,N/A,26.3012,Negative,N/A,N/A,N/A
60,F1,21.1384,25.8971,Positive,N/A,N/A,N/A
72,G1,N/A,26.3247,Negative,N/A,N/A,N/A
84,H1,21.1387,26.2765,Positive,N/A,N/A,N/A
1,A2,20.6485,25.0585,Positive,N/A,N/A,N/A
13,B2,N/A,26.2152,Negative,N/A,N/A,N/A


,Well,CT Value SARS-CoV-2,CT Value RNASE P,Result,REPEAT Ct VALUE SARS-CoV-2,REPEAT Ct VALUE RNASE P,REPEAT RESULT
0,A1,N/A,N/A,Negative,N/A,N/A,N/A
12,B1,29.7159,29.8258,Positive,N/A,N/A,N/A
24,C1,N/A,N/A,Negative,N/A,N/A,N/A
36,D1,29.6111,30.5403,Positive,N/A,N/A,N/A
48,E1,N/A,26.3254,Negative,N/A,N/A,N/A
60,F1,21.2861,26.1781,Positive,N/A,N/A,N/A
72,G1,39.9126,26.4274,REPEAT,REPEAT,REPEAT,REPEAT
84,H1,21.3649,26.1443,Positive,N/A,N/A,N/A
1,A2,22.0374,25.6399,Positive,N/A,N/A,N/A
13,B2,N/A,26.603,Negative,N/A,N/A,N/A


,Well,CT Value SARS-CoV-2,CT Value RNASE P,Result,REPEAT Ct VALUE SARS-CoV-2,REPEAT Ct VALUE RNASE P,REPEAT RESULT
0,A1,N/A,N/A,Negative,N/A,N/A,N/A
12,B1,30.1453,30.3614,Positive,N/A,N/A,N/A
24,C1,N/A,N/A,Negative,N/A,N/A,N/A
36,D1,29.7138,30.9073,Positive,N/A,N/A,N/A
48,E1,N/A,26.4787,Negative,N/A,N/A,N/A
60,F1,20.9284,25.729,Positive,N/A,N/A,N/A
72,G1,N/A,26.6614,Negative,N/A,N/A,N/A
84,H1,21.0347,26.0642,Positive,N/A,N/A,N/A
1,A2,20.6237,25.1461,Positive,N/A,N/A,N/A
13,B2,N/A,25.7542,Negative,N/A,N/A,N/A


,Well,CT Value SARS-CoV-2,CT Value RNASE P,Result,REPEAT Ct VALUE SARS-CoV-2,REPEAT Ct VALUE RNASE P,REPEAT RESULT
0,A1,N/A,N/A,Negative,N/A,N/A,N/A
12,B1,30.0634,30.0386,Positive,N/A,N/A,N/A
24,C1,N/A,N/A,Negative,N/A,N/A,N/A
36,D1,29.8069,30.9912,Positive,N/A,N/A,N/A
48,E1,N/A,26.3176,Negative,N/A,N/A,N/A
60,F1,21.2006,26.2161,Positive,N/A,N/A,N/A
72,G1,N/A,26.4465,Negative,N/A,N/A,N/A
84,H1,21.237,26.0451,Positive,N/A,N/A,N/A
1,A2,20.5913,25.3497,Positive,N/A,N/A,N/A
13,B2,N/A,26.1409,Negative,N/A,N/A,N/A


This is displaying 96 data, sorting by columns.

In [7]:
checkerBoardValidationMethod('AUTO CHECKER-96 BRAVO (1) -  Quantification Plate View Results.xlsx',
                            sortBy=['Well col', 'Well row'])

,Well,CT Value SARS-CoV-2,CT Value RNASE P,Result,REPEAT Ct VALUE SARS-CoV-2,REPEAT Ct VALUE RNASE P,REPEAT RESULT
0,A1,N/A,N/A,Negative,N/A,N/A,N/A
12,B1,33.3474,32.3648,Positive,N/A,N/A,N/A
24,C1,N/A,N/A,Negative,N/A,N/A,N/A
36,D1,32.6563,32.5033,Positive,N/A,N/A,N/A
48,E1,N/A,24.9734,Negative,N/A,N/A,N/A
60,F1,19.8782,25.8255,Positive,N/A,N/A,N/A
72,G1,N/A,25.0483,Negative,N/A,N/A,N/A
84,H1,19.8413,25.4304,Positive,N/A,N/A,N/A
1,A2,20.4211,25.4598,Positive,N/A,N/A,N/A
13,B2,N/A,24.1276,Negative,N/A,N/A,N/A
